# Using Resnet for chest X-ray Tuberculosis classification

In [1]:
import os
import os
import random
import argparse
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
import torch
from torch import nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm



In [2]:
# Define the data directory
data_dir = r'G:\deeplearning\exam1deeplearning\data\TB_Chest_Radiography_Database'
# Define model checkpoints directory
save_dir = "checkpoints"

In [3]:
# ---------------------------
# Reproducibility & device
# ---------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# ---------------------------
# Utility: metrics
# ---------------------------
def evaluate_model(model, loader):
    model.eval()
    y_true, y_probs, y_pred = [], [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            logits = model(xb).squeeze(-1).cpu()  # [batch]
            probs = torch.sigmoid(logits).numpy()
            preds = (probs >= 0.5).astype(int)
            y_probs.extend(probs.tolist())
            y_pred.extend(preds.tolist())
            y_true.extend(yb.numpy().tolist())
    acc = accuracy_score(y_true, y_pred)
    try:
        auc = roc_auc_score(y_true, y_probs)
    except Exception:
        auc = float("nan")
    cls_report = classification_report(y_true, y_pred, digits=4)
    return acc, auc, cls_report

In [5]:
set_seed(42)

# transforms
tf = transforms.Compose([
      transforms.RandomResizedCrop((224,224)),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
  ])

full_ds = datasets.ImageFolder(os.path.join(data_dir), transform=tf)
# Store class_to_idx before splitting
class_to_idx = full_ds.class_to_idx

train_len = int(len(full_ds)*0.8)
val_len = len(full_ds) - train_len
train_ds, val_ds = random_split(full_ds, [train_len, val_len], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=1)

In [6]:
# model:resnet50 -> single logit output
# using pretrained with weights='DEFAULT'
# training from scratch with weights=None
model = models.resnet50(weights=None)

for param in model.parameters():
      param.requires_grad = True  # fine-tune all (or set False to freeze)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # single logit
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

best_auc = 0.0
os.makedirs(save_dir, exist_ok=True)

for epoch in range(2):
    model.train()
    running_loss = 0.0
    for xb, yb in tqdm(train_loader):
        xb, yb = xb.to(device), yb.float().to(device)
        logits = model(xb).squeeze(-1)  # [batch]
        loss = criterion(logits, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    val_acc, val_auc, val_report = evaluate_model(model, val_loader)
    scheduler.step(val_loss := train_loss)  # or use val_auc etc
    print(f"[Epoch {epoch}] train_loss={train_loss:.4f} val_acc={val_acc:.4f} val_auc={val_auc:.4f} \n {val_report}")

    if val_auc > best_auc:
        best_auc = val_auc
        ckpt = os.path.join(save_dir, "tb_resnet50_best.pt")
        torch.save({"model_state": model.state_dict(), "class_to_idx": class_to_idx}, ckpt)
        print(f"  Saved best checkpoint to {ckpt}")

  # final eval
test_acc, test_auc, test_report = evaluate_model(model, val_loader)
print(f"Final val acc={test_acc:.4f}, auc={test_auc:.4f} \n {test_report}")

100%|██████████| 210/210 [00:41<00:00,  5.08it/s]


[Epoch 0] train_loss=0.2875 val_acc=0.8964 val_auc=0.9482 
               precision    recall  f1-score   support

           0     0.9734    0.8990    0.9347       693
           1     0.6500    0.8844    0.7493       147

    accuracy                         0.8964       840
   macro avg     0.8117    0.8917    0.8420       840
weighted avg     0.9168    0.8964    0.9023       840

  Saved best checkpoint to checkpoints\tb_resnet50_best.pt


100%|██████████| 210/210 [00:42<00:00,  4.98it/s]


[Epoch 1] train_loss=0.2106 val_acc=0.9119 val_auc=0.9431 
               precision    recall  f1-score   support

           0     0.9613    0.9307    0.9457       693
           1     0.7160    0.8231    0.7658       147

    accuracy                         0.9119       840
   macro avg     0.8386    0.8769    0.8558       840
weighted avg     0.9183    0.9119    0.9143       840

Final val acc=0.9179, auc=0.9417 
               precision    recall  f1-score   support

           0     0.9643    0.9351    0.9495       693
           1     0.7321    0.8367    0.7810       147

    accuracy                         0.9179       840
   macro avg     0.8482    0.8859    0.8652       840
weighted avg     0.9237    0.9179    0.9200       840



Mô hình được huấn luyện với 2 epoch trên tập dữ liệu phân loại nhị phân (2 lớp: 0 và 1).
Kết quả thể hiện hiệu suất cao và ổn định qua từng giai đoạn huấn luyện.

🔹 Kết quả Epoch 0

Train loss: 0.2875 → mô hình bắt đầu học tốt, sai số trung bình thấp.

Validation accuracy: 89.6%

AUC: 0.9482 → Khả năng phân biệt giữa hai lớp rất tốt.
| Lớp | Precision | Recall | F1-score |
| --- | --------- | ------ | -------- |
| 0   | 0.9734    | 0.8990 | 0.9347   |
| 1   | 0.6500    | 0.8844 | 0.7493   |

→ Lớp 0 (âm tính) chiếm đa số và được nhận diện rất tốt.
→ Lớp 1 (dương tính) có recall cao (mô hình bắt được hầu hết ca dương) nhưng precision thấp hơn, nghĩa là vẫn có một số dự đoán dương sai.

🔹 Kết quả Epoch 1

Train loss: 0.2106 → tiếp tục giảm, mô hình học ổn định hơn.

Validation accuracy: 91.2%

AUC: 0.9431 → gần như không giảm, vẫn duy trì khả năng phân biệt tốt.
| Lớp | Precision | Recall | F1-score |
| --- | --------- | ------ | -------- |
| 0   | 0.9613    | 0.9307 | 0.9457   |
| 1   | 0.7160    | 0.8231 | 0.7658   |

→ Hiệu suất lớp thiểu số (1) được cải thiện rõ rệt: recall giữ ở mức cao (82%), precision tăng từ 65% lên 71%.
→ Mô hình không bị overfitting, khả năng khái quát vẫn tốt.

🔹 Đánh giá cuối cùng

Final accuracy: 91.8%

Final AUC: 0.9417
| Lớp | Precision | Recall | F1-score |
| --- | --------- | ------ | -------- |
| 0   | 0.9643    | 0.9351 | 0.9495   |
| 1   | 0.7321    | 0.8367 | 0.7810   |

→ Mô hình đạt hiệu suất cao trên cả hai lớp, đặc biệt vẫn duy trì độ chính xác cao trên lớp thiểu số.
→ AUC > 0.94 cho thấy mô hình phân biệt hai lớp tốt, phù hợp với bài toán y sinh hoặc phát hiện bệnh có mất cân bằng dữ liệu.